<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [1]:
import pandas as pd
import seaborn
import warnings
import sklearn
from pymystem3 import Mystem
from pandas.core.common import SettingWithCopyWarning
warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)
import optuna

from statsmodels.tsa.seasonal import seasonal_decompose
from sklearn.model_selection import train_test_split
from sklearn.model_selection import TimeSeriesSplit 
from sklearn.preprocessing import StandardScaler


from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score, cross_validate
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_absolute_percentage_error

from tqdm import notebook

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D

from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import MinMaxScaler

In [2]:
# from keras.models import load_model
# model_loaded = load_model('modelC')

In [3]:
factory_2019 = pd.read_csv('C://Users//User//Desktop//123//Самарэнерго//Предприятие_2//csv файлы/2019 почасовки.csv', sep=';')
factory_2020 = pd.read_csv('C://Users//User//Desktop//123//Самарэнерго//Предприятие_2//csv файлы/2020 почасовки.csv', sep=';')
factory_2021 = pd.read_csv('C://Users//User//Desktop//123//Самарэнерго//Предприятие_2//csv файлы/2021 почасовки.csv', sep=';')
factory_2022 = pd.read_csv('C://Users//User//Desktop//123//Самарэнерго//Предприятие_2//csv файлы/2022 почасовки.csv', sep=';')
weather = pd.read_csv('C://Users//User//Desktop//123//Самарэнерго//Погодные_данные/weather_new.csv', sep=';', 
                      on_bad_lines='skip', decimal=",")

In [4]:
factory = factory_2019.append(factory_2020).append(factory_2021).append(factory_2022).reset_index(drop=True)
factory['datetime'] = factory['1']+' '+factory['2']
factory['datetime'] = pd.to_datetime(factory['datetime'], format='%d.%m.%Y %H:%M:%S')
factory = factory.rename(columns = {'3':'expenditure'})
factory = factory[['datetime', 'expenditure']]
factory = factory.set_index('datetime')

def cloud(a):
    a = str(a)
    a = a.replace('%', '')
    a = a.replace('.', '')
    if a == 'Облаков нет':
        return 0
    elif a =='Небо не видно из-за тумана и/или других метеорологических явлений':
        return 100
    elif '–' or ' ' in a:
        return a[:3]
    else:
        return a
    
def delited_1(a):
    a=str(a)
    a=a.replace('–', '')
    a=a.replace(' ', '')
    return a

def fill(a):
    if a == 'nan':
        return 100
    else:
        return a
    
weather = weather[['Местное время в Самаре','T']] #, 'U', 'Ff', 'N'
weather = weather.rename(columns = {'Местное время в Самаре':'datetime'})
weather['datetime'] = pd.to_datetime(weather['datetime'], format='%d.%m.%Y %H:%M')
weather = weather.set_index('datetime')
#weather['N'] = weather['N'].apply(cloud).apply(delited_1).apply(fill).astype('int')
weather = weather.fillna(method='bfill')
weather = weather.resample('1H').median()
weather = weather.reset_index()
for i in range(len(weather)-2):
    if weather['T'][i]==0:
        continue
    elif weather['T'][i]==len(weather)-1:
        break
    elif weather['T'][i]!='NaN':
        continue
    elif weather['T'][i]=='NaN' and weather['T'][i+2]=='NaN':
        weather['T'][i]= weather['T'][i-1]
    elif weather['T'][i]=='NaN' and weather['T'][i+1] =='NaN':
        weather['T'][i]= (weather['T'][i-1]+weather['T'][i+2])/2
    elif weather['T'][i]=='NaN' and weather['T'][i+1]!='NaN':
        weather['T'][i]= (weather['T'][i-1] +weather['T'][i+1])/2
weather = weather.set_index('datetime')
weather = weather.fillna(method='bfill')
weather = weather['2019-01-01 00:00:00':]

data = factory.join(weather, on='datetime')
def make_features(data, max_lag, rolling_mean_size):
    data['month'] = data.index.month
    data['day'] = data.index.day
    data['dayofweek'] = data.index.dayofweek
    data['is_weekend'] = data.dayofweek.isin([5,6])*1
    data['hour'] = data.index.hour
    data['expanding_window'] = data['expenditure'].expanding(3).mean()
    for lag in range(1, max_lag + 1):
        data['lag_{}'.format(lag)] = data['expenditure'].shift(lag)
    data['rolling_mean'] = data['expenditure'].shift().rolling(rolling_mean_size).mean()
    #data['kurtosis'] = data['rolling_mean'].shift().rolling(7).apply(lambda x: kurtosis(x))
    #data['peaks'] = data['rolling_mean'].shift().rolling(7).apply(lambda x: len(find_peaks(x)[0]))

# Обучение на всех данных

In [5]:
make_features(data, 24, 24)
data = data.dropna()
features = data.drop(['expenditure'], axis=1)
target = data['expenditure']

In [6]:
features

,T,month,day,dayofweek,is_weekend,hour,expanding_window,lag_1,lag_2,lag_3,...,lag_16,lag_17,lag_18,lag_19,lag_20,lag_21,lag_22,lag_23,lag_24,rolling_mean
datetime,,,,,,,,,,,,,,,,,,,,,
2019-01-02 00:00:00,-11.6,1,2,2,0,0,17991.160000,18471.0,18572.0,18919.0,...,17394.0,17834.0,18456.0,18433.0,18119.0,18001.0,18171.0,18139.0,18068.0,17963.250000
2019-01-02 01:00:00,-11.6,1,2,2,0,1,18018.038462,18661.0,18471.0,18572.0,...,17077.0,17394.0,17834.0,18456.0,18433.0,18119.0,18001.0,18171.0,18139.0,17987.958333
2019-01-02 02:00:00,-7.0,1,2,2,0,2,18039.518519,18690.0,18661.0,18471.0,...,16872.0,17077.0,17394.0,17834.0,18456.0,18433.0,18119.0,18001.0,18171.0,18010.916667
2019-01-02 03:00:00,-7.0,1,2,2,0,3,18062.714286,18598.0,18690.0,18661.0,...,16875.0,16872.0,17077.0,17394.0,17834.0,18456.0,18433.0,18119.0,18001.0,18028.708333
2019-01-02 04:00:00,-7.0,1,2,2,0,4,18102.448276,18689.0,18598.0,18690.0,...,17956.0,16875.0,16872.0,17077.0,17394.0,17834.0,18456.0,18433.0,18119.0,18057.375000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-05-31 19:00:00,22.8,5,31,1,0,19,45173.296618,27960.0,27552.0,30563.0,...,44900.0,44725.0,48123.0,44740.0,35991.0,45757.0,45831.0,44838.0,44024.0,38286.041667
2022-05-31 20:00:00,18.8,5,31,1,0,20,45172.818580,29406.0,27960.0,27552.0,...,42249.0,44900.0,44725.0,48123.0,44740.0,35991.0,45757.0,45831.0,44838.0,37676.958333
2022-05-31 21:00:00,18.8,5,31,1,0,21,45172.349529,30868.0,29406.0,27960.0,...,43709.0,42249.0,44900.0,44725.0,48123.0,44740.0,35991.0,45757.0,45831.0,37094.875000


In [7]:
scaler = MinMaxScaler(feature_range=(0, 1))
features = scaler.fit_transform(features)
n_steps = 32
n_features = 1
features = features.reshape((features.shape[0], features.shape[1], n_features))

In [8]:
try:
    #weather = weather['2019-01-01 00:00:00':]
    #factory_new = factory[:'2022-05-31']
    data_new = weather.join(factory, on='datetime')
    make_features(data_new, 24, 24)
    data_new = data_new['2019-01-03 00:00:00':]
    print()
    print('Введите, с какого дня начать прогноз, в формате "ГГГГ-ММ-ДД":')
    first_date = input()
    print()
    print('Введите, по какой день (включительно) сделать прогноз в формате "ГГГГ-ММ-ДД":')
    secord_date = input()
    
    time_for_predict = data_new[str(first_date+' 00:00:00'):str(secord_date+' 23:00:00')].index.to_list()
    for i in notebook.tqdm(time_for_predict):
        target_a = data_new.expenditure[i]
        features_a = data_new[str(i):str(i)].drop(['expenditure'], axis=1)
        features_a = scaler.transform(features_a)
        prediction = model_loaded.predict(features_a)
        data_new.expenditure[str(i)] = (float(prediction))
        make_features(data_new, 24, 24)
    prediction_april_test = data_new.expenditure[str(first_date+' 00:00:00'):]
    print('Предсказание завершено')
except:
    print()
    print('Ошибка. Проверьте, что данные ввода соответствует формату "ГГГГ-ММ-ДД"')


Введите, с какого дня начать прогноз, в формате "ГГГГ-ММ-ДД":
2022-06-01

Введите, по какой день (включительно) сделать прогноз в формате "ГГГГ-ММ-ДД":
2022-06-03


  0%|          | 0/72 [00:00<?, ?it/s]

1/1 [==============================] - 0s 15ms/step
Предсказание завершено


In [9]:
print('Куда сохранить файл? Введите адрес в формате',  
      r'C:\Users\User\Desktop\123\Самарэнерго\Предприятие_2\Предсказания\Предсказания тесты\нейросеть')
way_to_save = str(input())
prediction_april_test.to_csv(way_to_save) #и сохраняем
print()
print('Работа завершена. Данные сохранены в', way_to_save)

Куда сохранить файл? Введите адрес в формате C:\Users\User\Desktop\123\Самарэнерго\Предприятие_2\Предсказания\Предсказания тесты\нейросеть
C:\Users\User\Desktop\123\Самарэнерго\Предприятие_2\Предсказания\Предсказания тесты\нейросеть

Работа завершена. Данные сохранены в C:\Users\User\Desktop\123\Самарэнерго\Предприятие_2\Предсказания\Предсказания тесты\нейросеть
